<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/dreamstack_single_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
dreamstack_single: end-to-end dreamstack CLI (notebook-safe)

Subcommands:
  - run:       simulate survivors and frames (dummy workload, real artifacts)
  - visualize: summarize survivors into a CSV
  - compress:  compress frames into motifs (mock deterministic output)
  - replay:    prepare replay directories, optionally copying frames
  - fuse:      fuse top survivors into a combined config
  - pipeline:  run the full chain in the correct order

Notebook tips:
  - Use cli() to run without sys.exit, e.g.:
      from dreamstack_single import cli
      cli("run", steps=200, survivors=5, frames_per_survivor=12, out_dir="out_dreamstack", seed=42, verbose=1)
      cli("visualize", out_dir="out_dreamstack", top_n=3)
      cli("compress", out_dir="out_dreamstack", k=6, scope="local", top_n=3, seed=42)
      cli("replay", out_dir="out_dreamstack", top_n=3, copy=True)
      cli("fuse", out_dir="out_dreamstack", top_n=3, strategy="mix")
"""

from __future__ import annotations

import argparse
import csv
import json
import logging
import random
import shutil
import sys
from pathlib import Path
from typing import Iterable, List, Optional, Sequence


# ---------- Utilities ----------

def setup_logging(verbosity: int = 0) -> None:
    """
    verbosity: 0=WARNING, 1=INFO, 2+=DEBUG
    """
    level = logging.WARNING
    if verbosity >= 2:
        level = logging.DEBUG
    elif verbosity == 1:
        level = logging.INFO

    logging.basicConfig(
        level=level,
        format="%(asctime)s | %(levelname)-7s | %(name)s | %(message)s",
        datefmt="%H:%M:%S",
    )


def sanitize_argv(argv: Sequence[str]) -> List[str]:
    """
    Remove Jupyter/IPython kernel noise from argv so argparse stays clean.
    Safe in normal shells as well.
    """
    noisy_flags_prefixes = (
        "--ip", "--IPKernelApp", "--stdin", "--control", "--transport",
        "--hb", "--Session", "--kernel", "--ext", "--matplotlib",
        "--NotebookApp", "--HistoryManager",
    )
    cleaned: List[str] = []
    skip_next = False
    for a in argv:
        if skip_next:
            skip_next = False
            continue
        if a == "-f":
            # ipykernel passes -f <connection.json>
            skip_next = True
            continue
        if a.startswith("--f=") or a.startswith("--file="):
            continue
        if any(a.startswith(p) for p in noisy_flags_prefixes):
            continue
        # drop impl-specific launcher hints
        if "ipykernel" in a or "jupyter" in a:
            continue
        cleaned.append(a)
    return cleaned


def ensure_dir(p: Path) -> None:
    p.mkdir(parents=True, exist_ok=True)


def write_json(path: Path, obj) -> None:
    ensure_dir(path.parent)
    with path.open("w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2, sort_keys=True)


def write_csv(path: Path, headers: Iterable[str], rows: Iterable[Iterable]) -> None:
    ensure_dir(path.parent)
    with path.open("w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(list(headers))
        for r in rows:
            w.writerow(list(r))


# ---------- Core commands (lightweight, deterministic I/O) ----------

def cmd_run(steps: int, survivors: int, frames_per_survivor: int, out_dir: Path, seed: Optional[int]) -> int:
    log = logging.getLogger("run")
    rng = random.Random(seed)
    runs_dir = out_dir / "runs"
    ensure_dir(runs_dir)

    if steps <= 0 or survivors <= 0 or frames_per_survivor <= 0:
        log.error("steps, survivors, and frames_per_survivor must be positive")
        return 2

    log.info("Simulating %d survivors * %d frames (steps=%d) into %s",
             survivors, frames_per_survivor, steps, runs_dir)

    survivors_meta = []
    for s in range(survivors):
        sid = f"survivor_{s:03d}"
        sdir = runs_dir / sid / "frames"
        ensure_dir(sdir)

        # Generate dummy frames as text files, deterministically
        frame_scores = []
        for t in range(frames_per_survivor):
            score = rng.random()
            frame_scores.append(score)
            (sdir / f"frame_{t:04d}.txt").write_text(
                f"sid={sid} step={t} score={score:.6f}\n", encoding="utf-8"
            )

        # Aggregate simple survivor "fitness"
        fitness = sum(frame_scores) / len(frame_scores)
        survivors_meta.append({"id": sid, "fitness": fitness, "frames": frames_per_survivor})

    write_json(out_dir / "runs" / "survivors.json", survivors_meta)
    log.info("Wrote survivors.json with %d entries", len(survivors_meta))
    return 0


def cmd_visualize(out_dir: Path, top_n: int) -> int:
    log = logging.getLogger("visualize")
    runs_dir = out_dir / "runs"
    meta_path = runs_dir / "survivors.json"
    if not meta_path.exists():
        log.error("Missing %s. Run 'run' first.", meta_path)
        return 2

    survivors_meta = json.loads(meta_path.read_text(encoding="utf-8"))
    # Sort by fitness desc
    survivors_meta.sort(key=lambda x: x.get("fitness", 0.0), reverse=True)
    rows = []
    for i, m in enumerate(survivors_meta):
        rows.append([i, m["id"], m.get("fitness", 0.0), m.get("frames", 0)])

    write_csv(out_dir / "visualize" / "summary.csv",
              headers=["rank", "survivor_id", "fitness", "frames"],
              rows=rows)
    # Also store top list for downstream steps
    top = survivors_meta[:max(0, top_n)]
    write_json(out_dir / "visualize" / "top.json", top)
    log.info("Wrote visualization summary and top.json (top_n=%d)", top_n)
    return 0


def cmd_compress(out_dir: Path, k: int, scope: str, top_n: int, seed: Optional[int]) -> int:
    log = logging.getLogger("compress")
    vis_top = out_dir / "visualize" / "top.json"
    if not vis_top.exists():
        log.error("Missing %s. Run 'visualize' first.", vis_top)
        return 2

    rng = random.Random(seed)
    top = json.loads(vis_top.read_text(encoding="utf-8"))
    considered = top[:max(0, top_n)]
    motifs = []
    for i in range(max(0, k)):
        motifs.append({
            "id": f"motif_{i:03d}",
            "scope": scope,
            "weight": round(rng.uniform(0.1, 1.0), 6),
        })

    obj = {
        "considered_survivors": [m["id"] for m in considered],
        "k": k,
        "scope": scope,
        "motifs": motifs,
    }
    write_json(out_dir / "compress" / "motifs.json", obj)
    log.info("Compressed into %d motifs (scope=%s)", k, scope)
    return 0


def cmd_replay(out_dir: Path, top_n: int, copy: bool) -> int:
    log = logging.getLogger("replay")
    vis_top = out_dir / "visualize" / "top.json"
    runs_dir = out_dir / "runs"
    if not vis_top.exists():
        log.error("Missing %s. Run 'visualize' first.", vis_top)
        return 2

    top = json.loads(vis_top.read_text(encoding="utf-8"))
    selected = top[:max(0, top_n)]

    replay_dir = out_dir / "replay"
    ensure_dir(replay_dir)

    for m in selected:
        sid = m["id"]
        src_frames = runs_dir / sid / "frames"
        dst = replay_dir / sid
        ensure_dir(dst)
        # Always write a descriptor; optionally copy frames
        write_json(dst / "replay.json", {"survivor_id": sid, "frames_source": str(src_frames)})
        if copy:
            if not src_frames.exists():
                log.warning("No frames for %s at %s", sid, src_frames)
            else:
                dst_frames = dst / "frames"
                ensure_dir(dst_frames)
                count = 0
                for f in sorted(src_frames.glob("frame_*.txt")):
                    shutil.copy2(f, dst_frames / f.name)
                    count += 1
                log.debug("Copied %d frames for %s", count, sid)

    log.info("Prepared replay for %d survivors (copy=%s)", len(selected), copy)
    return 0


def cmd_fuse(out_dir: Path, top_n: int, strategy: str) -> int:
    log = logging.getLogger("fuse")
    vis_top = out_dir / "visualize" / "top.json"
    motifs_path = out_dir / "compress" / "motifs.json"
    if not vis_top.exists() or not motifs_path.exists():
        log.error("Missing visualize/compress outputs. Run 'visualize' and 'compress' first.")
        return 2

    top = json.loads(vis_top.read_text(encoding="utf-8"))
    motifs = json.loads(motifs_path.read_text(encoding="utf-8"))

    selected = top[:max(0, top_n)]
    fused = {
        "strategy": strategy,
        "from_survivors": [m["id"] for m in selected],
        "motifs_used": motifs.get("motifs", []),
    }
    write_json(out_dir / "fuse" / "fused_config.json", fused)
    log.info("Fused %d survivors using strategy=%s", len(selected), strategy)
    return 0


# ---------- Pipeline ----------

def pipeline(ns: argparse.Namespace) -> int:
    log = logging.getLogger("pipeline")
    log.info("Pipeline start")
    code = cmd_run(ns.steps, ns.survivors, ns.frames_per_survivor, Path(ns.out_dir), ns.seed)
    if code != 0:
        return code
    code = cmd_visualize(Path(ns.out_dir), ns.top_n)
    if code != 0:
        return code
    code = cmd_compress(Path(ns.out_dir), ns.k, ns.scope, ns.top_n, ns.seed)
    if code != 0:
        return code
    code = cmd_replay(Path(ns.out_dir), ns.top_n, ns.copy)
    if code != 0:
        return code
    code = cmd_fuse(Path(ns.out_dir), ns.top_n, ns.strategy)
    if code != 0:
        return code
    log.info("Pipeline complete")
    return 0


# ---------- Subcommand adapters ----------

def cmd_run_ns(ns: argparse.Namespace) -> int:
    return cmd_run(ns.steps, ns.survivors, ns.frames_per_survivor, Path(ns.out_dir), ns.seed)


def cmd_visualize_ns(ns: argparse.Namespace) -> int:
    return cmd_visualize(Path(ns.out_dir), ns.top_n)


def cmd_compress_ns(ns: argparse.Namespace) -> int:
    return cmd_compress(Path(ns.out_dir), ns.k, ns.scope, ns.top_n, ns.seed)


def cmd_replay_ns(ns: argparse.Namespace) -> int:
    return cmd_replay(Path(ns.out_dir), ns.top_n, ns.copy)


def cmd_fuse_ns(ns: argparse.Namespace) -> int:
    return cmd_fuse(Path(ns.out_dir), ns.top_n, ns.strategy)


# ---------- Parser ----------

def build_parser() -> argparse.ArgumentParser:
    p = argparse.ArgumentParser(
        prog="dreamstack_single",
        description="End-to-end dreamstack (notebook-safe).",
        formatter_class=argparse.ArgumentDefaultsHelpFormatter,
    )
    p.add_argument("-v", "--verbose", action="count", default=0,
                   help="Increase verbosity (-v for INFO, -vv for DEBUG)")

    sub = p.add_subparsers(dest="subcmd")

    # run
    prun = sub.add_parser("run", help="Run dream evolution")
    prun.add_argument("--steps", type=int, default=200, help="Simulation steps (placeholder)")
    prun.add_argument("--survivors", type=int, default=5, help="Number of survivors to produce")
    prun.add_argument("--frames-per-survivor", type=int, default=16, dest="frames_per_survivor",
                      help="Frames per survivor")
    prun.add_argument("--out-dir", type=str, default="out_dreamstack", dest="out_dir", help="Output directory")
    prun.add_argument("--seed", type=int, default=None, help="Random seed")
    prun.set_defaults(func=cmd_run_ns)

    # visualize
    pvis = sub.add_parser("visualize", help="Summarize survivors and candidates (CSV)")
    pvis.add_argument("--out-dir", type=str, default="out_dreamstack", dest="out_dir", help="Output directory")
    pvis.add_argument("--top-n", type=int, default=3, dest="top_n", help="Top survivors to carry forward")
    pvis.set_defaults(func=cmd_visualize_ns)

    # compress
    pcmp = sub.add_parser("compress", help="Compress frames into motifs")
    pcmp.add_argument("--out-dir", type=str, default="out_dreamstack", dest="out_dir", help="Output directory")
    pcmp.add_argument("--k", type=int, default=8, help="Number of motifs")
    pcmp.add_argument("--scope", type=str, choices=("local", "global"), default="local", help="Compression scope")
    pcmp.add_argument("--top-n", type=int, default=3, dest="top_n", help="Top survivors considered")
    pcmp.add_argument("--seed", type=int, default=None, help="Random seed")
    pcmp.set_defaults(func=cmd_compress_ns)

    # replay
    prep = sub.add_parser("replay", help="Prepare replays of top survivors")
    prep.add_argument("--out-dir", type=str, default="out_dreamstack", dest="out_dir", help="Output directory")
    prep.add_argument("--top-n", type=int, default=3, dest="top_n", help="Top survivors to replay")
    prep.add_argument("--copy", action="store_true", help="Copy frames into replay dirs")
    prep.set_defaults(func=cmd_replay_ns)

    # fuse
    pfuse = sub.add_parser("fuse", help="Fuse configs from top survivors")
    pfuse.add_argument("--out-dir", type=str, default="out_dreamstack", dest="out_dir", help="Output directory")
    pfuse.add_argument("--top-n", type=int, default=3, dest="top_n", help="Top survivors to fuse")
    pfuse.add_argument("--strategy", type=str, choices=("average", "best", "mix"), default="mix",
                       help="Fusion strategy")
    pfuse.set_defaults(func=cmd_fuse_ns)

    # pipeline
    ppipe = sub.add_parser("pipeline", help="Run full pipeline: run -> visualize -> compress -> replay -> fuse")
    ppipe.add_argument("--steps", type=int, default=200, help="Simulation steps (placeholder)")
    ppipe.add_argument("--survivors", type=int, default=5, help="Number of survivors to produce")
    ppipe.add_argument("--frames-per-survivor", type=int, default=16, dest="frames_per_survivor",
                       help="Frames per survivor")
    ppipe.add_argument("--out-dir", type=str, default="out_dreamstack", dest="out_dir", help="Output directory")
    ppipe.add_argument("--seed", type=int, default=None, help="Random seed")
    ppipe.add_argument("--top-n", type=int, default=3, dest="top_n", help="Top survivors to carry forward")
    ppipe.add_argument("--k", type=int, default=8, help="Number of motifs")
    ppipe.add_argument("--scope", type=str, choices=("local", "global"), default="local", help="Compression scope")
    ppipe.add_argument("--copy", action="store_true", help="Copy frames into replay dirs")
    ppipe.add_argument("--strategy", type=str, choices=("average", "best", "mix"), default="mix",
                       help="Fusion strategy")
    ppipe.set_defaults(func=pipeline)

    return p


# ---------- Entrypoints ----------

def main(argv: Optional[List[str]] = None, *, exit: bool = True) -> int:
    """
    Main entrypoint.
    - argv: arguments excluding program name. If None, uses sys.argv[1:].
    - exit: if True, calls sys.exit(code). If False, returns code.
    """
    raw_argv = sys.argv[1:] if argv is None else argv
    argv_clean = sanitize_argv(raw_argv)
    parser = build_parser()

    # If no subcommand, print help and return 0
    if not argv_clean or (argv_clean and argv_clean[0].startswith("-") and len(argv_clean) == 1):
        try:
            ns, _ = parser.parse_known_args(argv_clean)
            setup_logging(getattr(ns, "verbose", 0))
        except SystemExit:
            setup_logging(0)
        parser.print_help()
        if exit:
            sys.exit(0)
        return 0

    try:
        ns = parser.parse_args(argv_clean)
    except SystemExit as e:
        # Return code instead of exiting if exit=False
        if exit:
            raise
        return int(getattr(e, "code", 2) or 0)

    setup_logging(getattr(ns, "verbose", 0))

    if not hasattr(ns, "func"):
        parser.print_help()
        if exit:
            sys.exit(0)
        return 0

    code = int(ns.func(ns))
    if exit:
        sys.exit(code)
    return code


def cli(command: str, /, **kwargs) -> int:
    """
    Notebook-friendly helper:
      cli("run", steps=200, survivors=5, frames_per_survivor=12, out_dir="out_dreamstack", seed=42, verbose=1)
      cli("visualize", out_dir="out_dreamstack", top_n=3)
      cli("compress", out_dir="out_dreamstack", k=6, scope="local", top_n=3, seed=42)
      cli("replay", out_dir="out_dreamstack", top_n=3, copy=True)
      cli("fuse", out_dir="out_dreamstack", top_n=3, strategy="mix")
    Converts kwargs into CLI flags and avoids sys.exit.
    """
    args: List[str] = [str(command)]
    v = kwargs.pop("verbose", None)
    if isinstance(v, int) and v > 0:
        args.extend(["-" + "v" * min(v, 3)])
    for k, val in kwargs.items():
        flag = "--" + k.replace("_", "-")
        if isinstance(val, bool):
            if val:
                args.append(flag)
        elif isinstance(val, (list, tuple)):
            for item in val:
                args.extend([flag, str(item)])
        elif val is not None:
            args.extend([flag, str(val)])
    return main(args, exit=False)


if __name__ == "__main__":
    main()